# Kuala Lumpur Real Estate Price Prediction - Data Preparation

Use dataset published by Kaggle - https://www.kaggle.com/dragonduck/property-listings-in-kuala-lumpur - to train a simple deep learning model to predict prices for real estate in Kuala Lumpur.


This notebook contains the common data loading and preparation steps:
- load data from the input CSV
- do an assessment of the dataset to understand the number of distinct, missing, or invalid values by column


# Common imports and variables
Imports and variable definitions that are common to the entire notebook


In [1]:
!pip install requests
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime as dt
# common imports
import zipfile
import time
# import datetime, timedelta
import datetime
from datetime import datetime, timedelta
from datetime import date
from dateutil import relativedelta
from io import StringIO
import pandas as pd
import pickle
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from io import StringIO
import requests
import json
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import math
from subprocess import check_output
from IPython.display import display
import logging
import yaml
from collections import Counter
import re
import os
colab_path = "machine_learning_tabular_book/code/deep_learning_best_practices"


In [3]:
!pwd

/content


In [4]:
def get_config(config_file):
    ''' open config file with name config_file that contains parameters
    for this module and return Python object

    Args:
        config_file: filename containing config parameters

    Returns:
        config: Python dictionary with config parms from config file - dictionary


    '''
    current_path = os.getcwd()
    path_to_yaml = os.path.join(current_path, config_file)
    print("path_to_yaml " + path_to_yaml)
    try:
        with open(path_to_yaml, 'r') as c_file:
            config = yaml.safe_load(c_file)
        return config
    except Exception as error:
        print('Error reading the config file ' + str(error))

In [5]:
def print_config_values(config):
    for val in config:
        print("config value ",val," ",str(config[val]))

In [6]:
def set_current_directory():
  ''' check to see if the notebook is being run in Colab, and if so, set the current directory appropriately'''
  if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/machine_learning_tabular_book/code/deep_learning_best_practices/notebooks


# Load Data
- ingest CSV into a Pandas dataframe 

In [7]:
def get_path():
    ''' get the path for data files

    Returns:
        path: path for data directory

    '''
    rawpath = os.getcwd()
    # data is in a directory called "data" that is a sibling to the directory
    # containing the notebook
    path = os.path.abspath(os.path.join(rawpath, '..', 'data'))
    return path

In [8]:
def ingest_data(path,input_csv,pickled_input_dataframe,save_raw_dataframe,load_from_scratch):
    ''' load data into dataframe
    Args:
        path: path containing input file
        input_csv: input file name
        pickled_input_dataframe: pickled version of input file

    Returns:
        path: path for data directory
    '''
    if load_from_scratch:
        # if loading from scratch, the raw CSV file is expected to be in the data directory which is a sibling to the 
        # directory that contains this notebook
        unpickled_df = pd.read_csv(os.path.join(path,input_csv)) 
        if save_raw_dataframe:
            file_name = os.path.join(path,pickled_input_dataframe)
            print("file_name is ",file_name)
            unpickled_df.to_pickle(file_name)
    else:
        unpickled_df = pd.read_pickle(os.path.join(path,pickled_input_dataframe))
        logging.debug("reloader done")
    return(unpickled_df)

# Assess values
- assess columns for missing or invalid values

In [9]:
def not_in_list(x, list):
    ''' check if a value is in a list
    Args:
        x: value to check
        list: list in which to check for the value

    Returns:
        retur_val: 1 if value is in not in list, 0 otherwise
    '''
    if x in list:
        return_val = 0
    else:
        return_val = 1
    return(return_val)
    

In [10]:
def neg_val(x):
    ''' check if a value is in a list
    Args:
        x: value to check
    
    Returns:
        retur_val: 1 if value is negative, 0 otherwise
    '''
    if x >= 0:
        return_val = 0
    else:
        return_val = 1
    return(return_val)

In [11]:
def basic_assessment(df,columns,valid_values,non_neg_continuous):
    ''' assess the values in a dataframe
    Args:
        df: dataframe for assessment
        columns: dictionary of column names by category
        valid_values: dictionary of valid values for categorical columns with limited number of valid values
        non_neg_continuous: list of continuous columns with only non-negative values as valid
    '''
    for col in list(df):
        print("Missing values in ",col," ",str(df[col].isna().sum()))
        print("Distinct values in ",col," ",str(df[col].nunique()))
    # for categorical columns with a limited number of valid values, count the number of invalid values by column
    for col in valid_values:
        print("non-valid values in column ",col," ",str(df[col].apply(lambda x:not_in_list(x,valid_values[col])).sum()))
    # count non-numeric values in continuous columns
    for col in columns['continuous']:
        # mask = pd.to_numeric(df['Hours_Worked'], errors='coerce').isna()
        mask = pd.to_numeric(df[col], errors='coerce').isna()
        print("non-numeric values in continuous col ",col," ",str(mask.sum()))
        # if there are no non-numeric values in the column and it muast have non-negative values, count negative values
        if (mask.sum()==0) and (col in non_neg_continuous):
            print("negative values in colum ",col," ",str(df[col].apply(lambda x:neg_val(x)).sum()))
    

In [12]:
def out_of_range(x,max,min):
    ''' count whether a value is in a range
    Args:
        x: value to check in range
        max: top of the range to check
        min: bottom of the range to check
        
    Returns:
        ret_val: 1 if out of range, 0 otherwise
    '''
    if x > max or x < min:
        return_val = 1
    else:
        return_val = 0
    return(return_val)

In [13]:
def remove_currency_symbol(currency_string, input_string):
    ''' remove the currency symbol from a string
    Args:
        currency_string: currency symbol
        input_string: string from which to remove currency symbol
        
    Returns:
        output_string: input_string with currency symbol removed
    '''    
    #print("input_string is",input_string)
    #print(" on length ",len(input_string))
    output_string = re.sub(currency_string,'',input_string)
    return(output_string)

In [14]:
def remove_after_space(input_string):
  ''' remove everything after the first space in a string
  Args:
  input_string: string from which to remove content after space
  Returns:
  output_string: input_string with content removed
  '''    
  # remove leading and trailing spaces
  input_string = input_string.strip()
  # remove everything after internal spaces
  output_string = re.sub(r'\s* .*', '', input_string)
  output_string = re.sub(r'\([^)]*\)','',output_string)
  return(output_string)

In [15]:
def clean_up_size_col(df,clean_up_list):
    '''
    For the Size column in the KL real estate dataset:
    - lowercase values 
    - split the dimension values from the size type values
    - remove records with no numeric values
    - remove records with problematic values in clean_up_list
    Args:
      df: input dataframe
      clean_up_list: list of strings that, if a row contains them, the row should be removed from the dataset
    Returns:
      df: updated dataframe
    '''
    # remove rows with missing Size values
    df.dropna(subset=['Size'], inplace=True)

    # lowercase values in the Size column
    df['Size'] = df['Size'].str.lower()

    # split the Size column into two columns
    df[['Size_type','Size']] = df['Size'].str.split(':',expand=True)

    # replace missing values in the Size column with 0
    df['Size'] = df['Size'].fillna("0")
    
    # remove rows from Size that do not contain any digits
    df = df[df.Size.str.contains(r'\d')]
    # remove rows from Size that contain substrings from the clean_up_list
    for string in clean_up_list:
        df = df[~df.Size.str.contains(string,na=False)]
    # replace extraneous characters so that all Size entries are either numeric or of the form "numerica * numericb"
    df['Size'] = df['Size'].str.replace(',','').str.replace('`','').str.replace('@','x').str.replace('\+ sq. ft.','')
    df['Size'] = df['Size'].str.replace(' sq. ft.','').str.replace('sf sq. ft.','').str.replace('ft','').str.replace('sq','').str.replace("xx","*").str.replace("x ","*").str.replace(" x","*").str.replace("x","*").str.replace("X","*").replace('\'','')
    # remove extraneous characters following spaces
    df['Size'] = df['Size'].apply(lambda x: remove_after_space(x))
    # apply transformation to replace "numerica * numericb" with the result of the multiplication
    df['Size'] = df['Size'].apply(lambda x: eval(str(x)))
    return(df)

In [16]:
def clean_up_price_col(df):
  '''
    For the Price column in the KL real estate dataset:
    - remove rows where the Price value is missing
    - remove currency symbol from remaining rows
    - convert values to float 
    Args:
      df: input dataframe
      clean_up_list: list of strings that, if a row contains them, the row should be removed from the dataset
    Returns:
      df: updated dataframe
  '''
  df.dropna(subset=['Price'], inplace=True)
  # remove currency symbol from remaining rows
  df['Price'] = df['Price'].apply(lambda x: remove_currency_symbol("RM ",x))


  # convert Price column to float
  df['Price'] = pd.to_numeric(df['Price'].str.replace(',',''), errors='coerce')
  return(df)

In [33]:
def clean_up_rooms_col(df):
  '''
    For the Rooms column in the KL real estate dataset:
    - if there is an operation in the value, conduct that operation
    - convert the value from string to integer
    Args:
      df: input dataframe
    Returns:
      df: updated dataframe
  '''
  df['Rooms'] = df['Rooms'].apply(lambda x: eval(x))
  df['Rooms'] = pd.to_numeric(df['Rooms'])
  return(df)

# Master cell
This cell contains calls to the other functions in this notebook to complete the data preparation

In [17]:
# master cell to call the other functions
# get the path for data files
set_current_directory()
path = get_path()
print("path is ",path)
config = get_config('data_preparation_config.yml')
logging.getLogger().setLevel(logging.WARNING)
logging.warning("logging check")
#print_config_values(config)
# load dataframe and, if parameter set, save CSV file as a pickled dataframe
df = ingest_data(path,config['file_names']['input_csv'],config['file_names']['pickled_input_dataframe'],config['general']['save_raw_dataframe'],config['general']['load_from_scratch'])
print("shape ",df.shape)
'''
# get basic assessment information for the dataframe
basic_assessment(df,config['columns'],config['valid_values'],config['non_negative_continuous'])
# get assessment for geospatial information
geo_assessment(df,config['bounding_box'])
'''
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/machine_learning_tabular_book/code/deep_learning_best_practices/notebooks
path is  /content/drive/MyDrive/machine_learning_tabular_book/code/deep_learning_best_practices/data
path_to_yaml /content/drive/MyDrive/machine_learning_tabular_book/code/deep_learning_best_practices/notebooks/data_preparation_config.yml
shape  (53883, 8)


,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,"KLCC, Kuala Lumpur","RM 1,250,000",2+1,3.0,2.0,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,"Damansara Heights, Kuala Lumpur","RM 6,800,000",6,7.0,NaN,Bungalow,Land area : 6900 sq. ft.,Partly Furnished
2,"Dutamas, Kuala Lumpur","RM 1,030,000",3,4.0,2.0,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
3,"Cheras, Kuala Lumpur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Bukit Jalil, Kuala Lumpur","RM 900,000",4+1,3.0,2.0,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished


In [18]:
df.describe()

,Bathrooms,Car Parks
count,51870.000000,36316.000000
mean,3.073434,2.007848
std,1.631079,1.310427
min,1.000000,1.000000
25%,2.000000,1.000000
50%,2.000000,2.000000
75%,4.000000,2.000000
max,20.000000,30.000000


In [19]:
# complete clean up of Price and Size columns
df = clean_up_price_col(df)
clean_up_list = ["-","\+",'\'','\~',"xx","sf","acre","#"]
df = clean_up_size_col(df,clean_up_list)
print("shape ",df.shape)
df.head()

<ipython-input-15-92801a20170b>:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Size'] = df['Size'].str.replace(',','').str.replace('`','').str.replace('@','x').str.replace('\+ sq. ft.','')
<ipython-input-15-92801a20170b>:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Size'] = df['Size'].str.replace(' sq. ft.','').str.replace('sf sq. ft.','').str.replace('ft','').str.replace('sq','').str.replace("xx","*").str.replace("x ","*").str.replace(" x","*").str.replace("x","*").str.replace("X","*").replace('\'','')


shape  (52309, 9)


,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,Size_type
0,"KLCC, Kuala Lumpur",1250000,2+1,3.0,2.0,Serviced Residence,1335.0,Fully Furnished,built-up
1,"Damansara Heights, Kuala Lumpur",6800000,6,7.0,NaN,Bungalow,6900.0,Partly Furnished,land area
2,"Dutamas, Kuala Lumpur",1030000,3,4.0,2.0,Condominium (Corner),1875.0,Partly Furnished,built-up
4,"Bukit Jalil, Kuala Lumpur",900000,4+1,3.0,2.0,Condominium (Corner),1513.0,Partly Furnished,built-up
5,"Taman Tun Dr Ismail, Kuala Lumpur",5350000,4+2,5.0,4.0,Bungalow,7200.0,Partly Furnished,land area


In [34]:
df = clean_up_rooms_col(df)
df.head()

TypeError: ignored

In [20]:
df.describe()

,Price,Bathrooms,Car Parks,Size
count,5.230900e+04,50499.000000,35347.000000,5.230900e+04
mean,1.977360e+06,3.080259,2.011147,2.848243e+03
std,1.310709e+07,1.634402,1.315736,4.933040e+04
min,3.080000e+02,1.000000,1.000000,0.000000e+00
25%,5.880000e+05,2.000000,1.000000,1.008000e+03
50%,9.982100e+05,3.000000,2.000000,1.412000e+03
75%,2.000000e+06,4.000000,2.000000,2.400000e+03
max,1.980000e+09,20.000000,30.000000,1.100000e+07


In [21]:
path

'/content/drive/MyDrive/machine_learning_tabular_book/code/deep_learning_best_practices/data'

In [22]:
# if specified, save the transformed dataset as a pickle file
if config['general']['save_transformed_dataframe']:
    print("path is ",path)
    file_name = os.path.join(path,config['file_names']['pickled_output_dataframe'])
    print("file_name is ",file_name)
    df.to_pickle(file_name)

path is  /content/drive/MyDrive/machine_learning_tabular_book/code/deep_learning_best_practices/data
file_name is  /content/drive/MyDrive/machine_learning_tabular_book/code/deep_learning_best_practices/data/kl_real_estate_output.pkl


In [23]:
df.columns

Index(['Location', 'Price', 'Rooms', 'Bathrooms', 'Car Parks', 'Property Type',
       'Size', 'Furnishing', 'Size_type'],
      dtype='object')

In [24]:
# dataset columns by type
# continuous
# categorical
categorical_columns = df.select_dtypes(include=['object']).columns
continuous_columns = continuous_columns = df.select_dtypes(include=['float', 'int']).columns

In [25]:
df['Location'].value_counts()

Mont Kiara, Kuala Lumpur                           5795
KLCC, Kuala Lumpur                                 5132
Cheras, Kuala Lumpur                               4314
Jalan Klang Lama (Old Klang Road), Kuala Lumpur    2780
Setapak, Kuala Lumpur                              2601
                                                   ... 
Santuari Park Pantai, Kuala Lumpur                    1
Bukit  Persekutuan, Kuala Lumpur                      1
Wangsa Melawati, Kuala Lumpur                         1
Taman Yarl OUG, Kuala Lumpur                          1
Kuala Lumpur, Kuala Lumpur                            1
Name: Location, Length: 109, dtype: int64

In [26]:
print("continuous columns: ",continuous_columns)
print("categorical columns: ",categorical_columns)

continuous columns:  Index(['Price', 'Bathrooms', 'Car Parks', 'Size'], dtype='object')
categorical columns:  Index(['Location', 'Rooms', 'Property Type', 'Furnishing', 'Size_type'], dtype='object')


In [28]:
result = eval("1")
result

1